Part 2: digit autoencoder 

loading prerequisites and dataset

again getting help from ChatGPT and Isaiah Hazelwood

In [2]:
from nn import nn 
from nn import io 
from nn import preprocess
import sklearn
import numpy as np
import matplotlib.pyplot as plt 

In [4]:
#loading digits data
import sklearn.datasets

#this is a small dataset of handwritten digit images used for classification, great for learning and testing ML models quickly without needing to download anything
digits_data = sklearn.datasets.load_digits()

In [5]:
#reshape data for processing
#there are 1797 images, each image is a 8x8 grayscale image. it's flattened into a 1D array of 64 pixel values... so you get shape 1797, 64. I am reshaping because my model expects input to be a vector... -> shape = features, 1 #a column vector
digits_input = digits_data["data"].reshape(1797, 64, 1)
digits_label = digits_data["target"].reshape(1797, 1, 1)

In [ ]:
#split data into training and validation sets, a common split is 60% for training, 20% for validation and 20% for test
"""
np.random.permutation shuffles the rows of digits_input. important so that the model doesn't see samples in any biased order e.g. first 0s then 1s etc. 
/16.0 scales pixel values from 0-16 to 0-1, so normalized. """
digits_input_permuted = np.random.permutation(digits_input) / 16.0  

#let's say the number of samples is 1797. so I am slicing the data into training set first 70% from index 0 to 1256
digits_input_train = digits_input_permuted[:digits_input.shape[0] * 7 // 10, :]

#now the validation set takes the last 30%, so everyithing from index 1258 to the end
digits_input_val = digits_input_permuted[digits_input.shape[0] * 7 // 10:, :]